## 센텐스피스(SentencePiece)

BPE를 포함하여 기타 서브워드 토크나이징 알고리즘을 내장한 패키지

일반적으로 선택할 수 있는 최선의 선택 중 하나

BPE, Unigram Language Model Tokenizer 등이 구현되어 있음

사전 토큰화 작업없이 단어 분리 토큰화를 수행

따라서 사전 토큰화 작업(pretokenization)없이 전처리를 하지 않은 데이터(raw data)에 바로 단어 분리 토크나이저를 사용할 수 있음

언어에 종속되지 않음

## 센텐스피스 예제

In [38]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

### IMDB

In [39]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x217680fe2c0>)

In [40]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [41]:
print(len(train_df))

50000


In [42]:
# 센텐스피스의 입력으로 사용하기 위해서 데이터프레임을 txt 파일로 저장
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [43]:
# 센텐스피스로 단어 집합과 각 단어에 고유한 정수를 부여
# input : 학습시킬 파일
# model_prefix : 만들어질 모델 이름
# vocab_size : 단어 집합의 크기
# model_type : 사용할 모델 (unigram(default), bpe, char, word)
# max_sentence_length: 문장의 최대 길이
# pad_id, pad_piece: pad token id, 값
# unk_id, unk_piece: unknown token id, 값
# bos_id, bos_piece: begin of sentence token id, 값
# eos_id, eos_piece: end of sequence token id, 값
# user_defined_symbols: 사용자 정의 토큰
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')
# vocab 생성이 완료되면 imdb.model, imdb.vocab 파일 두개가 생성

In [44]:
# vocab 파일에서 학습된 서브워드들을 확인할 수 있음
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
4060,▁scientist,-4057
980,▁comes,-977
1509,▁To,-1506
3258,▁impression,-3255
473,▁Wh,-470
4405,ean,-4402
1725,▁mot,-1722
4040,▁Dick,-4037
699,ript,-696
3024,▁critic,-3021


In [45]:
len(vocab_list)

5000

In [46]:
# model 파일을 로드하여 단어 시퀀스를 정수 시퀀스로 바꾸는 인코딩 작업이나 반대로 변환하는 디코딩 작업을 할 수 있음
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

In [47]:
lines = [
    "I didn't at all think of it this way.",
    "I have waited a long time for someone to film"
]

In [48]:
for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))  # 문장을 입력하면 서브 워드 시퀀스로 변환
    print(sp.encode_as_ids(line))  # 문장을 입력하면 정수 시퀀스로 변환
    print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 624, 4950, 4926, 139, 170, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 142, 1364, 1121, 4, 668, 285, 93, 1079, 33, 91]



In [49]:
# 단어 집합의 크기 확인
sp.GetPieceSize()

5000

In [50]:
# 정수로부터 맵핑되는 서브 워드로 변환
sp.IdToPiece(430)

'▁character'

In [51]:
# 서브워드로부터 맵핑되는 정수로 변환
sp.PieceToId('▁character')

430

In [52]:
# 정수 시퀀스로부터 문장으로 변환
sp.DecodeIds([41, 142, 1364, 1121, 4, 668, 285, 93, 1079, 33, 91])

'I have waited a long time for someone to film'

In [53]:
# 서브워드 시퀀스로부터 문장으로 변환
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

### 네이버 영화 리뷰

In [54]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x217680ffc40>)

In [55]:
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [56]:
print(len(naver_df))

200000


In [57]:
# null값 확인
print(naver_df.isnull().values.any())

True


In [58]:
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [59]:
print(len(naver_df))

199992


In [60]:
# 텍스트 파일로 변환
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [61]:
# 단어 집합을 생성
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [62]:
# subwords를 확인
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,ᄏᄏ,-6


In [63]:
len(vocab_list)

5000

In [64]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [65]:
lines = [
    "뭐 이딴 것도 영화냐.",
    "진짜 최고의 영화입니다 ㅋㅋ",
]

In [66]:
for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[136, 970, 1299, 2593, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 204, 825, 121]



In [67]:
sp.GetPieceSize()

5000

In [68]:
sp.IdToPiece(4)

'영화'

In [69]:
sp.PieceToId('영화')

4

In [70]:
sp.DecodeIds([54, 204, 825, 121])

'진짜 최고의 영화입니다 ᄏᄏ'

In [71]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])

'진짜 최고의 영화입니다 ᄏᄏ'

In [72]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 204, 825, 121]
